In [60]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [61]:
# 위노그라드 알고리즘 설정
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

# 데이터 로드

In [62]:
rootPath = './vision/'

In [63]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [64]:
#imageGenerator = ImageDataGenerator(
#    rotation_range=20,
#    horizontal_flip=True,
#    preprocessing_function = get_random_eraser(v_l=0, v_h=255),
#    validation_split=.1)
    
    


In [65]:
imageGenerator = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        validation_split=.1,
        horizontal_flip=True,
        fill_mode='nearest')


In [66]:
trainGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='training'
)

Found 1157 images belonging to 3 classes.


In [67]:
validationGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='validation'
)

Found 127 images belonging to 3 classes.


# 모델 구성

In [68]:
model = Sequential()
model.add(ResNet50(include_top=True, weights=None, input_shape=(100, 100, 3), classes=3))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 3)                 23593859  
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [69]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['acc'],
)

In [70]:
epochs = 150
history = model.fit_generator(
    trainGen, 
    epochs=epochs,
    steps_per_epoch=trainGen.samples / epochs, 
    validation_data=validationGen,
    validation_steps=trainGen.samples / epochs,
)

Epoch 1/150
7/7 [==============================] - 11s 566ms/step - loss: 1.1324 - acc: 0.5039 - val_loss: 480.4903 - val_acc: 0.6535
Epoch 2/150
7/7 [==============================] - 2s 236ms/step - loss: 0.3839 - acc: 0.7578
Epoch 3/150
7/7 [==============================] - 2s 216ms/step - loss: 0.2666 - acc: 0.8008
Epoch 4/150
7/7 [==============================] - 2s 242ms/step - loss: 0.2718 - acc: 0.8242
Epoch 5/150
7/7 [==============================] - 2s 249ms/step - loss: 0.2871 - acc: 0.7891
Epoch 6/150
7/7 [==============================] - 2s 209ms/step - loss: 0.2397 - acc: 0.8125
Epoch 7/150
7/7 [==============================] - 2s 210ms/step - loss: 0.1876 - acc: 0.8945
Epoch 8/150
7/7 [==============================] - 2s 231ms/step - loss: 0.1576 - acc: 0.9214
Epoch 9/150
7/7 [==============================] - 2s 217ms/step - loss: 0.2039 - acc: 0.8711
Epoch 10/150
7/7 [==============================] - 2s 235ms/step - loss: 0.1879 - acc: 0.8789
Epoch 11/150
7/7 [=

# 평가

In [71]:
testGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'test_set'),
    target_size=(100, 100),
)

Found 15 images belonging to 3 classes.


In [72]:
pred = model.predict(testGen)

In [73]:
img_list = testGen.filenames

In [74]:
pred_list = []
for i in range(len(pred)):
    pred_list.append(np.argmax(pred[i]))

In [75]:
df = pd.DataFrame(list(zip(img_list, pred_list)), columns =['Name', 'pred'])

In [76]:
df.to_csv('submission.csv', index=False)